In [ ]:
# !/usr/bin/env python3
# -*- coding: utf-8 -*-


import re
from datetime import datetime
import time
import os
import tensorflow as tf
import csv
import pandas as pd
from pandas import Series, DataFrame
from numpy import array
import numpy as np
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,2,3,4"
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
config = tf.ConfigProto()
config.gpu_options.allow_growth = True



INPUT_NODE = 2304
OUTPUT_NODE = 7
LAYER1_NODE = 4096
LAYER2_NODE = 512
LAYER3_NODE = 64
BATCH_SIZE = 1024
LEARNING_RATE_BASE = 0.01
LEARNING_RATE_DECAY = 0.992
REGULARIZER = 0.0001
STEPS = 100000
MOVING_AVERAGE_DECAY = 0.99
TEST_INTERVAL_SECS = 1
MODEL_SAVE_PATH="check_point_next"
MODEL_NAME="data_model"



def get_weight(shape, regularizer):
    w = tf.Variable(tf.truncated_normal(shape, stddev=0.1,mean=0))
    #损失函数loss含正则化regularization
    if regularizer != None: tf.add_to_collection('losses', tf.contrib.layers.l2_regularizer(regularizer)(w))
    return w


def get_bias(shape):
    b = tf.Variable(tf.zeros(shape))
    return b


def conv2d(x, W):
    # stride[1, x_movement, y_movement, 1]
    # Must have strides[0] = strides[3] =1，意思是不对样本个数和channel进行卷积
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding="SAME")  # padding="SAME"用零填充边界


def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")


def forward(x, regularizer):
 
    x = tf.reshape(x, [-1, 48, 48, 1])
    
    ## convl layer ##
    W_conv1 = get_weight([5,5,1,128],None) # kernel 5*5, channel is 1, out size 32
    b_conv1 = get_bias([128])
    h_conv1 = tf.nn.relu(conv2d(x,W_conv1) + b_conv1)  # output size 28*28*32
    h_pool1 = max_pool_2x2(h_conv1)                          # output size 14*14*32
    
    ## conv2 layer ##
    W_conv2 = get_weight([5,5,128,64],None) # kernel 5*5, in size 32, out size 64
    b_conv2 = get_bias([64])
    h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2) + b_conv2)  # output size 14*14*64
    h_pool2 = max_pool_2x2(h_conv2)                          # output size 7*7*64
    
    ## conv3 layer ##
    W_conv3 = get_weight([5,5,64,32],None) # kernel 5*5, in size 32, out size 64
    b_conv3 = get_bias([32])
    h_conv3 = tf.nn.relu(conv2d(h_pool2,W_conv3) + b_conv3)  # output size 14*14*64
    h_pool3 = max_pool_2x2(h_conv3)                          # output size 7*7*64
    
    ## funcl layer ##
    W_fc1 = get_weight([1*6*6*32, 576], regularizer)
    b_fc1 = get_bias([576])
    
    h_pool2_flat = tf.reshape(h_pool3, [-1, 1*6*6*32])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
    h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob = 0.5)
    
    ## func2 layer ##
    W_fc2 = get_weight([576, 128], regularizer)
    b_fc2 = get_bias([128])
    y1 = tf.nn.relu(tf.matmul(h_fc1_drop,W_fc2)+b_fc2)
    
    W_fc3 = get_weight([128, OUTPUT_NODE], regularizer)
    b_fc3 = get_bias([OUTPUT_NODE])
    y = (tf.matmul(y1,W_fc3)+b_fc3)
    tf.add_to_collection('pred_network', y)  # 用于加载模型获取要预测的网络结构
    return W_fc1,b_fc1,h_fc1,W_fc2,b_fc2,y


def backward(new_data):
    step_train = 0
    keep_prob = tf.placeholder(tf.float32)


    
    np.random.shuffle(new_data)


    
    x = tf.placeholder(tf.float32, [None, INPUT_NODE],name='x')
    y_ = tf.placeholder(tf.float32, [None, OUTPUT_NODE])

    w1, b1, y1, w2, b2,y = forward(x, REGULARIZER)
    global_step = tf.Variable(0, trainable=False)

    #损失函数loss含正则化regularization
    ce = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.argmax(y_, 1))
    cem = tf.reduce_mean(ce)
    loss = cem + tf.add_n(tf.get_collection('losses'))

    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    correct_prediction = tf.cast(correct_prediction, tf.float32)
    accuracy = tf.reduce_mean(correct_prediction)
    
    #指数衰减学习率
    learning_rate = tf.train.exponential_decay(
        LEARNING_RATE_BASE,
        global_step,
        len(new_data) / BATCH_SIZE,
        LEARNING_RATE_DECAY,
        staircase=True)



    train_step = tf.train.AdamOptimizer(learning_rate).minimize(loss, global_step=global_step)

    #滑动平均
    ema = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)
    ema_op = ema.apply(tf.trainable_variables())

    with tf.control_dependencies([train_step, ema_op]):
        train_op = tf.no_op(name='train')

    saver = tf.train.Saver()

    with tf.Session(config = config) as sess:
        init_op = tf.global_variables_initializer()
        sess.run(init_op)

        ckpt = tf.train.get_checkpoint_state(MODEL_SAVE_PATH)
        if ckpt and ckpt.model_checkpoint_path:
            saver.restore(sess, ckpt.model_checkpoint_path)

        for i in range(STEPS):
            #每次读入BATCH_SIZE组数据和标签

            if(step_train > len(new_data)):
                step_train = 0

                continue
            else:
                xs = new_data[step_train:step_train+BATCH_SIZE,:2304]

                ys = new_data[step_train:step_train+BATCH_SIZE,2304:]
            step_train = step_train + BATCH_SIZE


            _, loss_value, step, accuracy_train = sess.run([train_op, loss, global_step, accuracy], feed_dict={x: xs, y_: ys, keep_prob:1})

            if i % 100 == 0:

                print("%s after %d training step(s), loss on training batch is %.4f, learning rate is %f, accuracy is %f." % (datetime.now(),step, loss_value,learning_rate.eval(),accuracy_train))
                #print("w1 = ", (sess.run(w1)))

                saver.save(sess, os.path.join(MODEL_SAVE_PATH, MODEL_NAME), global_step=global_step)
    print("finish！")


def test(new_data):
    

    np.random.shuffle(new_data)
    new_data = new_data[0:1500]
    with tf.Graph().as_default() as g:
        x = tf.placeholder(tf.float32, [None, INPUT_NODE])
        y_ = tf.placeholder(tf.float32, [None, OUTPUT_NODE])
        keep_prob = tf.placeholder(tf.float32)
        w1, b1, y1, w2, b2, y = forward(x, None)

        xs = new_data[:, :2304]

        ys = new_data[:, 2304:]

        ema = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY)
        ema_restore = ema.variables_to_restore()
        saver = tf.train.Saver(ema_restore)

        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

        while True:
            with tf.Session(config=config) as sess:
                ckpt = tf.train.get_checkpoint_state(MODEL_SAVE_PATH)
                if ckpt and ckpt.model_checkpoint_path:
                    saver.restore(sess, ckpt.model_checkpoint_path)
                    global_step = ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1]
                    accuracy_score = sess.run(accuracy, feed_dict={x: xs, y_: ys, keep_prob : 0.5})
                    print("After %s training step(s), test accuracy = %g" % (global_step, accuracy_score))
                else:
                    print('No checkpoint file found')
                    return
            time.sleep(TEST_INTERVAL_SECS)




def main():
    
    train=True
    
    if train == True: 
        print("train!")
        data = pd.read_csv("fer2013.csv",low_memory=False).values
        data = data.astype(np.float32)  # change to numpy array and float32
        data = data / 255.0
        
        print("finish fer2013!")
        
        print("数组元素总数：",data.size)      #打印数组尺寸，即数组元素总数  
        print("数组形状：",data.shape)         #打印数组形状 
        print("数组的维度数目",data.ndim)      #打印数组的维度数目
        
        backward(data)
        
    else:
        print("test!")
        data = pd.read_csv("test.csv",low_memory=False).values
        data = data.astype(np.float32)  # change to numpy array and float32
        data = data / 255.0
        print("finish test!")

        print("数组元素总数：",data.size)      #打印数组尺寸，即数组元素总数  
        print("数组形状：",data.shape)         #打印数组形状 
        print("数组的维度数目",data.ndim)      #打印数组的维度数目
        
        test(data)

if __name__ == '__main__':

    main()

    


/home/multiphase/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


train!
finish fer2013!
数组元素总数： 66344188
数组形状： (28708, 2311)
数组的维度数目 2
INFO:tensorflow:Restoring parameters from check_point_next/data_model-1
2019-04-17 19:54:04.589375 after 2 training step(s), loss on training batch is 200.8303, learning rate is 0.010000, accuracy is 0.161133.
2019-04-17 19:54:23.537144 after 99 training step(s), loss on training batch is 2.0252, learning rate is 0.009762, accuracy is 0.263672.
2019-04-17 19:54:42.247466 after 196 training step(s), loss on training batch is 1.9907, learning rate is 0.009529, accuracy is 0.260742.
2019-04-17 19:55:00.958070 after 292 training step(s), loss on training batch is 2.0132, learning rate is 0.009228, accuracy is 0.212891.
2019-04-17 19:55:20.057120 after 389 training step(s), loss on training batch is 1.9151, learning rate is 0.009008, accuracy is 0.268555.
2019-04-17 19:55:39.217627 after 486 training step(s), loss on training batch is 1.9047, learning rate is 0.008724, accuracy is 0.284180.
2019-04-17 19:55:57.881558 afte

2019-04-17 20:12:43.878891 after 5706 training step(s), loss on training batch is 0.4040, learning rate is 0.001958, accuracy is 0.926758.
2019-04-17 20:13:02.539719 after 5802 training step(s), loss on training batch is 0.3864, learning rate is 0.001912, accuracy is 0.947266.
2019-04-17 20:13:21.266182 after 5899 training step(s), loss on training batch is 0.3699, learning rate is 0.001851, accuracy is 0.935547.
2019-04-17 20:13:40.108794 after 5996 training step(s), loss on training batch is 0.3444, learning rate is 0.001807, accuracy is 0.951172.
2019-04-17 20:13:58.697442 after 6092 training step(s), loss on training batch is 0.3420, learning rate is 0.001750, accuracy is 0.946289.
2019-04-17 20:14:17.921813 after 6189 training step(s), loss on training batch is 0.3146, learning rate is 0.001708, accuracy is 0.961914.
2019-04-17 20:14:36.948030 after 6286 training step(s), loss on training batch is 0.3101, learning rate is 0.001654, accuracy is 0.957031.
2019-04-17 20:14:55.594734 

2019-04-17 20:31:22.985472 after 11409 training step(s), loss on training batch is 0.1365, learning rate is 0.000383, accuracy is 0.984375.
2019-04-17 20:31:42.194708 after 11506 training step(s), loss on training batch is 0.1279, learning rate is 0.000371, accuracy is 0.991211.
2019-04-17 20:32:00.845902 after 11602 training step(s), loss on training batch is 0.1261, learning rate is 0.000363, accuracy is 0.991211.
2019-04-17 20:32:19.939564 after 11699 training step(s), loss on training batch is 0.1251, learning rate is 0.000351, accuracy is 0.993164.
2019-04-17 20:32:39.382774 after 11796 training step(s), loss on training batch is 0.1319, learning rate is 0.000343, accuracy is 0.990234.
2019-04-17 20:32:57.971052 after 11892 training step(s), loss on training batch is 0.1306, learning rate is 0.000332, accuracy is 0.989258.
2019-04-17 20:33:17.083195 after 11989 training step(s), loss on training batch is 0.1249, learning rate is 0.000324, accuracy is 0.988281.
2019-04-17 20:33:36.

2019-04-17 20:50:07.367781 after 17112 training step(s), loss on training batch is 0.0941, learning rate is 0.000074, accuracy is 0.990234.
2019-04-17 20:50:26.665430 after 17209 training step(s), loss on training batch is 0.0931, learning rate is 0.000073, accuracy is 0.991211.
2019-04-17 20:50:45.817459 after 17306 training step(s), loss on training batch is 0.0957, learning rate is 0.000070, accuracy is 0.993164.
2019-04-17 20:51:04.661158 after 17402 training step(s), loss on training batch is 0.0885, learning rate is 0.000069, accuracy is 0.991211.
2019-04-17 20:51:23.904176 after 17499 training step(s), loss on training batch is 0.0848, learning rate is 0.000067, accuracy is 0.997070.
2019-04-17 20:51:43.097325 after 17596 training step(s), loss on training batch is 0.0922, learning rate is 0.000065, accuracy is 0.991211.
2019-04-17 20:52:02.020577 after 17692 training step(s), loss on training batch is 0.0876, learning rate is 0.000063, accuracy is 0.993164.
2019-04-17 20:52:21.

...

2019-04-17 22:04:52.196943 after 39926 training step(s), loss on training batch is 0.0838, learning rate is 0.000000, accuracy is 0.993164.
2019-04-17 22:05:10.839364 after 40022 training step(s), loss on training batch is 0.0772, learning rate is 0.000000, accuracy is 0.994141.
2019-04-17 22:05:30.008060 after 40119 training step(s), loss on training batch is 0.0735, learning rate is 0.000000, accuracy is 0.995117.
2019-04-17 22:05:49.053097 after 40216 training step(s), loss on training batch is 0.0795, learning rate is 0.000000, accuracy is 0.995117.
2019-04-17 22:06:07.358949 after 40312 training step(s), loss on training batch is 0.0809, learning rate is 0.000000, accuracy is 0.993164.
2019-04-17 22:06:26.546836 after 40409 training step(s), loss on training batch is 0.0826, learning rate is 0.000000, accuracy is 0.992188.
2019-04-17 22:06:45.577777 after 40506 training step(s), loss on training batch is 0.0792, learning rate is 0.000000, accuracy is 0.995117.
2019-04-17 22:07:04.

2019-04-17 22:23:34.035470 after 45629 training step(s), loss on training batch is 0.0759, learning rate is 0.000000, accuracy is 0.997070.
2019-04-17 22:23:53.091526 after 45726 training step(s), loss on training batch is 0.0790, learning rate is 0.000000, accuracy is 0.995117.
2019-04-17 22:24:11.924845 after 45822 training step(s), loss on training batch is 0.0750, learning rate is 0.000000, accuracy is 0.997070.
2019-04-17 22:24:31.254774 after 45919 training step(s), loss on training batch is 0.0757, learning rate is 0.000000, accuracy is 0.998047.
2019-04-17 22:24:50.376633 after 46016 training step(s), loss on training batch is 0.0816, learning rate is 0.000000, accuracy is 0.995117.
2019-04-17 22:25:09.402285 after 46112 training step(s), loss on training batch is 0.0739, learning rate is 0.000000, accuracy is 0.998047.
2019-04-17 22:25:28.533941 after 46209 training step(s), loss on training batch is 0.0804, learning rate is 0.000000, accuracy is 0.992188.
2019-04-17 22:25:47.

2019-04-17 22:41:53.113887 after 51332 training step(s), loss on training batch is 0.0746, learning rate is 0.000000, accuracy is 0.995117.
2019-04-17 22:42:10.653595 after 51429 training step(s), loss on training batch is 0.0782, learning rate is 0.000000, accuracy is 0.996094.
2019-04-17 22:42:28.247803 after 51526 training step(s), loss on training batch is 0.0815, learning rate is 0.000000, accuracy is 0.992188.
2019-04-17 22:42:45.395549 after 51622 training step(s), loss on training batch is 0.0799, learning rate is 0.000000, accuracy is 0.993164.
2019-04-17 22:43:02.757269 after 51719 training step(s), loss on training batch is 0.0798, learning rate is 0.000000, accuracy is 0.996094.
2019-04-17 22:43:20.292567 after 51816 training step(s), loss on training batch is 0.0941, learning rate is 0.000000, accuracy is 0.987305.
2019-04-17 22:43:37.377707 after 51912 training step(s), loss on training batch is 0.0815, learning rate is 0.000000, accuracy is 0.993164.
2019-04-17 22:43:54.

2019-04-17 22:58:58.951250 after 57036 training step(s), loss on training batch is 0.0757, learning rate is 0.000000, accuracy is 0.997070.
2019-04-17 22:59:16.301069 after 57132 training step(s), loss on training batch is 0.0764, learning rate is 0.000000, accuracy is 0.995117.
2019-04-17 22:59:33.664982 after 57229 training step(s), loss on training batch is 0.0799, learning rate is 0.000000, accuracy is 0.995117.
2019-04-17 22:59:51.164691 after 57326 training step(s), loss on training batch is 0.0779, learning rate is 0.000000, accuracy is 0.996094.
2019-04-17 23:00:08.253931 after 57422 training step(s), loss on training batch is 0.0753, learning rate is 0.000000, accuracy is 0.997070.
2019-04-17 23:00:25.613637 after 57519 training step(s), loss on training batch is 0.0721, learning rate is 0.000000, accuracy is 0.998047.
2019-04-17 23:00:43.108793 after 57616 training step(s), loss on training batch is 0.0811, learning rate is 0.000000, accuracy is 0.994141.
2019-04-17 23:01:00.

2019-04-17 23:16:05.687044 after 62739 training step(s), loss on training batch is 0.0743, learning rate is 0.000000, accuracy is 0.998047.
2019-04-17 23:16:23.291907 after 62836 training step(s), loss on training batch is 0.0847, learning rate is 0.000000, accuracy is 0.992188.
2019-04-17 23:16:40.340577 after 62932 training step(s), loss on training batch is 0.0788, learning rate is 0.000000, accuracy is 0.995117.
2019-04-17 23:16:57.821690 after 63029 training step(s), loss on training batch is 0.0787, learning rate is 0.000000, accuracy is 0.995117.
2019-04-17 23:17:15.291862 after 63126 training step(s), loss on training batch is 0.0812, learning rate is 0.000000, accuracy is 0.995117.
2019-04-17 23:17:32.583215 after 63222 training step(s), loss on training batch is 0.0775, learning rate is 0.000000, accuracy is 0.996094.
2019-04-17 23:17:50.410931 after 63319 training step(s), loss on training batch is 0.0756, learning rate is 0.000000, accuracy is 0.995117.
2019-04-17 23:18:07.